In [ ]:
TUGAS BESAR PROJECT DASAR KECERDASAN ARTIFICIAL

In [13]:
import pandas as pd
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
# Load the dataset
df = pd.read_csv("hotel_bookings.csv")

# Select relevant columns
df = df[['lead_time', 'adr', 'total_of_special_requests', 'booking_changes', 'is_canceled']]

# Handle missing values (if any)
df.dropna(inplace=True)

# Clip outliers for fuzzy logic (optional but helps fuzzy control)
df['lead_time'] = df['lead_time'].clip(0, 500)
df['adr'] = df['adr'].clip(0, 400)
df['total_of_special_requests'] = df['total_of_special_requests'].clip(0, 5)
df['booking_changes'] = df['booking_changes'].clip(0, 9)

# Optional: Display summary
print(df.describe())

           lead_time            adr  total_of_special_requests  \
count  119390.000000  119390.000000              119390.000000   
mean      103.755583     101.786073                   0.571363   
std       105.798008      48.138316                   0.792798   
min         0.000000       0.000000                   0.000000   
25%        18.000000      69.290000                   0.000000   
50%        69.000000      94.575000                   0.000000   
75%       160.000000     126.000000                   1.000000   
max       500.000000     400.000000                   5.000000   

       booking_changes    is_canceled  
count    119390.000000  119390.000000  
mean          0.219943       0.370416  
std           0.630146       0.482918  
min           0.000000       0.000000  
25%           0.000000       0.000000  
50%           0.000000       0.000000  
75%           0.000000       1.000000  
max           9.000000       1.000000  


In [15]:
# Define fuzzy variables
lead_time = ctrl.Antecedent(np.arange(0, 501, 1), 'lead_time')
adr = ctrl.Antecedent(np.arange(0, 401, 1), 'adr')
special_requests = ctrl.Antecedent(np.arange(0, 6, 1), 'special_requests')
booking_changes = ctrl.Antecedent(np.arange(0, 10, 1), 'booking_changes')
cancellation_risk = ctrl.Consequent(np.arange(0, 101, 1), 'cancellation_risk')

# Membership functions
lead_time.automf(3)
adr.automf(3)
special_requests.automf(3)
booking_changes.automf(3)

# Manual membership functions
lead_time['short'] = fuzz.trimf(lead_time.universe, [0, 0, 150])
lead_time['medium'] = fuzz.trimf(lead_time.universe, [100, 250, 400])
lead_time['long'] = fuzz.trimf(lead_time.universe, [300, 500, 500])

adr['low'] = fuzz.trimf(adr.universe, [0, 0, 100])
adr['medium'] = fuzz.trimf(adr.universe, [75, 150, 250])
adr['high'] = fuzz.trimf(adr.universe, [200, 400, 400])

special_requests['low'] = fuzz.trimf(special_requests.universe, [0, 0, 1])
special_requests['medium'] = fuzz.trimf(special_requests.universe, [1, 2, 3])
special_requests['high'] = fuzz.trimf(special_requests.universe, [2, 5, 5])

booking_changes['low'] = fuzz.trimf(booking_changes.universe, [0, 0, 2])
booking_changes['medium'] = fuzz.trimf(booking_changes.universe, [1, 3, 5])
booking_changes['high'] = fuzz.trimf(booking_changes.universe, [4, 10, 10])

cancellation_risk['low'] = fuzz.trimf(cancellation_risk.universe, [0, 0, 40])
cancellation_risk['medium'] = fuzz.trimf(cancellation_risk.universe, [30, 50, 70])
cancellation_risk['high'] = fuzz.trimf(cancellation_risk.universe, [60, 100, 100])

# Fuzzy rules
rule1 = ctrl.Rule(lead_time['long'] & adr['high'], cancellation_risk['high'])
rule2 = ctrl.Rule(lead_time['short'] & special_requests['high'], cancellation_risk['low'])
rule3 = ctrl.Rule(booking_changes['high'], cancellation_risk['high'])

# Control System
cancellation_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
cancellation_sim = ctrl.ControlSystemSimulation(cancellation_ctrl)

# Function to predict cancellation risk
def predict_cancellation_risk(lt, rate, requests, changes):
    cancellation_sim.input['lead_time'] = lt
    cancellation_sim.input['adr'] = rate
    cancellation_sim.input['special_requests'] = requests
    cancellation_sim.input['booking_changes'] = changes
    cancellation_sim.compute()
    return cancellation_sim.output['cancellation_risk']